In [1]:
!pip install --force-reinstall transformers torch scikit-learn gensim datasets -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.20.0 requires torch==2.5.0, but you have torch 2.6.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Import libraries
import os
import json
import pandas as pd
import numpy as np
import nltk
import gensim
import re
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec

# Download NLTK resources
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\limyi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\limyi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\limyi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Mount your drive to access dataset from folder
from google.colab import drive
drive.mount("/content/drive")

ModuleNotFoundError: No module named 'google'

In [ ]:
# Load dataset (change MyDrive to Shared if it's not the same on yours)
train_df = pd.read_json("drive/MyDrive/CS4248_proj/Sarcasm_Headlines_Dataset.json", lines=True)

# Rename columns if necessary
train_df.rename(columns={"headline": "text", "is_sarcastic": "label"}, inplace=True)

# Preprocessing function
# def preprocess_text(text):
#     text = text.lower()  # Lowercasing
#     text = re.sub(r'\W', ' ', text)  # Remove non-alphanumeric characters
#     tokens = word_tokenize(text)  # Tokenization
#     tokens = [word for word in tokens if word.isalpha()]  # Remove numbers and punctuation
#     tokens = [word for word in tokens if word not in stopwords.words('english')]  # Stopword removal
#     lemmatizer = WordNetLemmatizer()
#     tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
#     return tokens

# # Apply preprocessing
# train_df['processed_text'] = train_df['text'].apply(preprocess_text)

# Split into train, test, validation (80% train, 20% validation from train)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['text'], train_df['label'], test_size=0.2, random_state=42)


# Convert processed text to list format for Word2Vec training
# train_sentences = train_texts.tolist()

# Train Word2Vec embeddings
# w2v_model = Word2Vec(sentences=train_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Save Word2Vec model
# w2v_model.save("word2vec.model")

# Convert tokenized words to vectors
# def text_to_vector(tokens, model):
#     vectors = [model.wv[word] for word in tokens if word in model.wv]
#     return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Apply word2vec transformation
# train_vectors = np.array([text_to_vector(tokens, w2v_model) for tokens in train_texts])
# val_vectors = np.array([text_to_vector(tokens, w2v_model) for tokens in val_texts])

# Summary
# print("Train samples:", len(train_vectors))
# print("Validation samples:", len(val_vectors))
# print("Word2Vec model trained and saved as 'word2vec.model'")

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

# Convert your data into a Hugging Face Dataset
train_data = {"text": train_texts, "label": train_labels}
val_data = {"text": val_texts, "label": val_labels}

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)

# Load BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Remove original text column
train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])

# Rename labels column for compatibility
train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")

# Set dataset format to PyTorch tensors
train_dataset.set_format("torch")
val_dataset.set_format("torch")


Map:   0%|          | 0/21367 [00:00<?, ? examples/s]

Map:   0%|          | 0/5342 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

# Load pre-trained BERT model (2 classes for binary classification)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-17-0338c3808d35>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.118000,0.260469
2,0.095800,0.322166
3,0.000500,0.418355


TrainOutput(global_step=8013, training_loss=0.17782298737399896, metrics={'train_runtime': 6233.1327, 'train_samples_per_second': 10.284, 'train_steps_per_second': 1.286, 'total_flos': 1.686568175963136e+16, 'train_loss': 0.17782298737399896, 'epoch': 3.0})

In [ ]:
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Get raw predictions from Trainer
predictions = trainer.predict(val_dataset)

# Extract logits and true labels
logits = predictions.predictions
labels = predictions.label_ids

# Convert logits to predicted class (0 or 1)
preds = torch.argmax(torch.tensor(logits), axis=-1).numpy()

# Compute accuracy, precision, recall, and F1-score
acc = accuracy_score(labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")

# Print evaluation results
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Accuracy: 0.9291
Precision: 0.9331
Recall: 0.9032
F1-score: 0.9179


In [ ]:
print(logits)

[[-4.3794003  3.6866612]
 [ 4.4344325 -4.1013885]
 [ 4.3581057 -4.094697 ]
 ...
 [ 2.5682642 -2.0835938]
 [-4.3539405  3.5912895]
 [-4.3797984  3.6862805]]


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)  # Convert logits to class predictions
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Run evaluation
trainer.evaluate()


{'eval_loss': 0.41835543513298035,
 'eval_runtime': 144.83,
 'eval_samples_per_second': 36.885,
 'eval_steps_per_second': 4.612,
 'epoch': 3.0}

In [ ]:
trainer.save_model("drive/MyDrive/CS4248_proj/bert_weights")